In [ ]:
# %pip install pandas

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.3 MB 11.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.3 MB 13.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.3 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 15.4 MB/s eta 0:00:00
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- --

In [3]:
import os
import json
import pandas as pd

In [9]:
from collections import defaultdict

merged_data = []
# Create a mapping from article_id to its real_text_id
real_map = {f"article_{str(row['id']).zfill(4)}": row['real_text_id'] for _, row in df.iterrows()}

# Group texts by article_id
grouped = defaultdict(list)
for entry in all_data:
    grouped[entry['article_id']].append(entry['text'])

# Build the merged structure
for article_id, texts in grouped.items():
    merged_data.append({
        "file_data": texts,
        "real": real_map.get(article_id, None)
    })

df_pair = pd.DataFrame({
    "text 1": [item["file_data"][0] for item in merged_data],
    "text 2": [item["file_data"][1] for item in merged_data],
    "real": [item["real"] for item in merged_data]
})
df_pair

,text 1,text 2,real
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,1
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,2
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,1
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,2
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...,2
...,...,...,...
90,A main focus of modern cosmology is to underst...,A key focus of modern cosmology is to understa...,2
91,"APEX, as its name suggests, serves as a guide ...","APEX, as its name suggests, serves as a guide ...",1
92,FORS1 and FORS2 are early instruments of the V...,FORS1 and FORS2 are early instruments of the V...,2
93,The observations of the Pluto-Charon system an...,The observations of the Pluto-Charon binary an...,2
